# Better Way 43 재사용 가능한 try/finally 동작을 만들려면 contextlib와 with문을 고려하자

* https://github.com/KangJungHwa/Effective-Python/blob/master/item_43.py
* https://github.com/shoark7/Effective-Python/blob/master/files/BetterWay43_UseContextlib.md



In [0]:
import logging
from pprint import pprint
from sys import stdout as STDOUT
from threading import Lock

* 파이썬의 with 문은 코드를 특별한 컨텍스트에서 실행함을 나타내는데 사용한다.
* 아래 두 개 코드는 동일한 역할을 한다.

In [2]:
# 잠금이 설정되어 있는 동안만 들여쓴 코드 실행 V1
lock = Lock()
with lock:
    print('Lock is held')

Lock is held


In [3]:
# 잠금이 설정되어 있는 동안만 들여쓴 코드 실행 V2
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()

Lock is held


## contextmanager

* 내장 모듈 contextlib를 사용하면, 객체와 함수를 with문에 사용할 수 있게 만들기 쉽다.
* contextlib 모듈은 간단한 함수를 with 문에 사용할 수 있게 해주는 contextmanager 데코레이터를 포함한다. 
* contextmanager 데코레이터를 이용하는 방법이 `__enter__, __exit__`라는 특별한 메소드를 담은 새 클래스를 정의하는 표준 방법보다 훨씬 쉽다.
<hr/>

예제:
*  로그는 상황에 따라 다양한 수준의 로그를 남기는데 가끔씩은 코드의 특정 영역에 더 많은 디버깅 로그를 넣고 싶다고 해보자. 
* 여기서는 로깅 심각성 수준(severity level) 두 개로 로그를 남기는 함수를 정의한다.

In [22]:
def my_function():
    logging.debug("Some bug happended")
    # logging.warning("w")
    logging.error("Something very bad;")
    logging.debug("Another small bug got caught!")

my_function()    

ERROR:root:Something very bad;


* 이 'contextmanager' 데코레이터에서는 yield 지점이 'with' 블록의 내용이 실행되는 지점이다.
* with 블록에서 일어나는 모든 예외를 yield 표현식이 다시 일으키므로 헬퍼 함수로 처리할 수 있다(BW40 참조).

In [0]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield # with 블록이 실행되는 지점
    finally:
        logger.setLevel(old_level)


In [16]:
# with 문 안에서는 debug 로그까지 찍힘
with debug_logging(logging.DEBUG):
    print('Inside:')
    my_function()

print('After:')
my_function()

DEBUG:root:Some bug happended
ERROR:root:Something very bad;
DEBUG:root:Another small bug got caught!
ERROR:root:Something very bad;


Inside:
After:


## with 타깃 사용하기(as)
* with 문에 전달되는 컨텍스트 매니저에서 객체를 반환할 수도 있다.
* 이 객체는 복합문의 as 부분에 있는 지역 변수에 할당된다.
* 이 기능을 이용하면, with 블록 안에 있는 코드에서 직접 컨텍스트와 상호작용이 가능하다.
* 아래와 같은 파일 핸들링 할 때가 가장 자주 접하는 예시.

In [0]:
with open("somefile.txt", 'w') as f:
  # f.write('test')
  pass

* 아까 로깅예시를 발전시켜 보면 아래처럼.
* 함수에서 as 타겟에 값을 제공하려면, 컨텍스트 매니저에서 yield를 사용하여 값을 넘겨주면 된다.

In [0]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

In [18]:
# 'my-log'라는 로거의 로깅 레벨을 디버그로 변경한 logger 
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug("This is my debug message!")
    logging.debug("This will not be printed")

DEBUG:my-log:This is my debug message!


In [19]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print


# 핵심 정리
* with 문을 이용하면 try / finally 블록의 로직을 재사용할 수 있고, 코드를 깔끔하게 만들 수 있다.
* 내장 모듈 contextlib의 contextmanager 데코레이터를 이용하면 직접 작성한 함수를 with 문에서 쉽게 사용할 수 있다.
* 컨텍스트 매니저에서 넘겨준 값은 with 문의 as 부분에 할당된다. 컨텍스트 매니저에서 값을 반환하는 방법은 코드에서 특별한 컨텍스트에 직접 접근하려는 경우에 유용하다.